In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def preprocess(image_path):
    # Read and normalize image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256,256))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image/255.0 # Normalize to [0,1]

In [ ]:
def trimmed_median_filter(image, window_size=3, d=2):
    # Reduce salt-and-pepper noise
    padded = cv2.copyMakeBorder(image, d, d, d, d, cv2.BORDER_REFLECT)
    filtered = cv2.medianBlur(padded, window_size)
    return filtered[d:-d, d:-d]

In [4]:
# Colore corection using gray world algorithm
def gray_world_balance(image):
    avg = np.mean(image, axis=(0,1))
    gray = np.mean(avg)
    scale = gray/avg
    return np.clip(image * scale, 0, 1)

In [5]:
# Contrast Enhancement using CLAHE
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createClahe(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply((l*255).astype(np.uint8))/255.0
    lab = cv2.merge((cl,a,b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

In [6]:
# Haze Removal using Laplacian Decomposition
def laplacian_haze_removal(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    laplacian = cv2.normalize(laplacian, None, 0, 1, cv2.NORM_MINMAX)
    enhanced = image - laplacian[..., np.newaxis] * 0.5  # Adjust haze strength
    return np.clip(enhanced, 0, 1)

In [8]:
# Example usage
image = preprocess("dataset/Paired/underwater_imagenet/trainA/n01496331_49.jpg")
filtered = trimmed_median_filter(image)
color_balanced = gray_world_balance(filtered)
contrast_enhanced = apply_clahe(color_balanced)
final_image = laplacian_haze_removal(contrast_enhanced)

# Plot results
fig, axes = plt.subplots(1, 5, figsize=(20, 5))
titles = ["Original", "Noise Reduced", "Color Balanced", "CLAHE", "Haze Removed"]
for ax, img, title in zip(axes, [image, filtered, color_balanced, contrast_enhanced, final_image], titles):
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
plt.show()

error: OpenCV(4.6.0) /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/imgproc/src/median_blur.simd.hpp:975: error: (-210:Unsupported format or combination of formats)  in function 'medianBlur'
